<a href="https://colab.research.google.com/github/sharvayuzade/Green_AI_Workshop/blob/main/Forest_Fire_Detection_Using_Satellite_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog #provides a dialog box to select files.
from PIL import Image, ImageTk #ImageTk - Converts images for display in Tkinter GUIs
import numpy as np

In [ ]:
# Set up directories
train_dir = r"Datasets\Forest fire classification data\train"
valid_dir = r"Datasets\Forest fire classification data\valid"
test_dir = r"Datasets\Forest fire classification data\test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [ ]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)

Epoch 1/10
946/946 [==============================] - 618s 653ms/step - loss: 0.1909 - accuracy: 0.9301 - val_loss: 0.1399 - val_accuracy: 0.9489
Epoch 2/10
946/946 [==============================] - 480s 507ms/step - loss: 0.1702 - accuracy: 0.9379 - val_loss: 0.2198 - val_accuracy: 0.9081
Epoch 3/10
946/946 [==============================] - 254s 268ms/step - loss: 0.1568 - accuracy: 0.9434 - val_loss: 0.1505 - val_accuracy: 0.9433
Epoch 4/10
946/946 [==============================] - 275s 290ms/step - loss: 0.1436 - accuracy: 0.9487 - val_loss: 0.1225 - val_accuracy: 0.9540
Epoch 5/10
946/946 [==============================] - 286s 302ms/step - loss: 0.1286 - accuracy: 0.9529 - val_loss: 0.1368 - val_accuracy: 0.9519
Epoch 6/10
946/946 [==============================] - 280s 296ms/step - loss: 0.1173 - accuracy: 0.9557 - val_loss: 0.1120 - val_accuracy: 0.9616
Epoch 7/10
946/946 [==============================] - 284s 300ms/step - loss: 0.1095 - accuracy: 0.9598 - val_loss: 0.1145 -

In [ ]:
#save the model
model.save("ffd_model.h5")

In [ ]:
import tkinter as tk
from tkinter import filedialog

from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model

In [ ]:
# Load the model
model = load_model("ffd_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


In [ ]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()